# Evaluation

## Data Pre-processing

先從置於同目錄下的 proc/ 讀取處理過的字幕資料，並利用 *jieba* 進行斷詞

In [17]:
from sklearn.metrics import f1_score
from utils import read_csvdir, sep_train_test, accuracy
from simple_nb import SimpleNB
from binary_nb import BinaryNB
from tfidf_nb import tfidfNB

data_arr = read_csvdir('./proc', details=True)

Read 2944 lines from Inferno_2016.csv
Read 3088 lines from Wonder_Woman_2017.csv
Read 4437 lines from Inception_2010.csv
Read 4485 lines from Spider-Man_Homecoming_2017.csv
Read 3104 lines from Baby_Driver_2017.csv


將所有訓練資料 shuffle，且依照 9:1 分為訓練資料與測試資料

In [4]:
train_arr, test_arr = sep_train_test(data_arr, ratio=0.1)
test_Y, test_X = zip(*test_arr)

print('# of training data =', len(train_arr))
print('# of test data     =', len(test_arr))

# of training data = 16253
# of test data     = 1805


## Cross-validation and Test

![cross-validation](https://raw.githubusercontent.com/ritchieng/machine-learning-dataschool/master/images/07_cross_validation_diagram.png)

In [23]:
def cross_val(model_class, data, cv=5):
    scores = []
    nb_seg = len(data) // cv

    print('Cross Validation')
    for i in range(cv):
        dev_data = data[i*nb_seg : (i+1)*nb_seg]
        train_data = data[: i*nb_seg] + data[(i+1)*nb_seg :]
        
        dev_Y, dev_X = zip(*dev_data)
        
        model = model_class()
        model.train(train_data)
        preds = model.predict(dev_X)
        score = f1_score(dev_Y, preds, average='macro')
        scores.append(score)

    print('  f1_score = [{}]'.format(', '.join(['{:4f}'.format(s) for s in scores])))
    print('  {}-fold cross-validation f1_score = {:.4f}'.format(cv, sum(scores)/cv))

def test(model_class, train_data, test_X, test_Y):
    model = model_class()
    model.train(train_data)
    preds = model.predict(test_X)
    
    print('\nTest')
    print('  accuracy        = {:.4f}'.format(accuracy(test_Y, preds)))
    print('  f1_score(micro) = {:.4f}'.format(f1_score(test_Y, preds, average='micro')))
    print('  f1_score(macro) = {:.4f}'.format(f1_score(test_Y, preds, average='macro')))

### Naive Bayes

In [25]:
cross_val(SimpleNB, train_arr, cv=2)
test(SimpleNB, train_arr, test_X, test_Y)

Cross Validation
  f1_score = [0.626305, 0.634632]
  2-fold cross-validation f1_score = 0.6305

Test
  accuracy        = 0.6327
  f1_score(micro) = 0.6327
  f1_score(macro) = 0.6275


### Binarized Naive Bayes

In [7]:
cross_val(BinaryNB, train_arr, cv=5)
test(BinaryNB, train_arr, test_X, test_Y)

Cross Validation
  f1_score = [0.618067, 0.624193, 0.622820, 0.629280, 0.639446]
  5-fold cross-validation f1_score = 0.6268

Test
  accuracy        = 0.6343
  f1_score(micro) = 0.6343
  f1_score(macro) = 0.6189


### TF-IDF Naive Bayes

In [24]:
cross_val(tfidfNB, train_arr, cv=2)
test(tfidfNB, train_arr, test_X, test_Y)

Cross Validation
  f1_score = [0.616320, 0.628998]
  2-fold cross-validation f1_score = 0.6227

Test
  accuracy        = 0.6382
  f1_score(micro) = 0.6382
  f1_score(macro) = 0.6307
